# Example: Find the normal modes for a two-mass-three-spring system

Consider two masses connected to three springs as follows:

<center><img src='https://raw.githubusercontent.com/autofeedback-exercises/exercises/testpip/MTH2031/project/2mass3springs.png' height=300 /></center>

We have already seen how to set up and solve the equations of motion for this system to determine how the masses move as a function of time. Here we will determine the so-called normal modes for the motion.

---

## Setup

First, let's import all of the libraries we will need, and define the constants and variables we already know about.

In [1]:
import sympy as sy
import numpy as np
from scipy.linalg import eig
import matplotlib.pyplot as plt

t, k1, k2, k3, m1, m2 = sy.symbols('t, k1, k2, k3, m1, m2')

Note that in principle with two particles we have six degrees of freedom, $x_1, y_1, z_1$ and $x_2, y_2, z_2$. However, for now we will restrict ourselves to motion in one direction, as per the diagram given, by setting $y_1 = z_1 = y_2 = z_2 = 0$. Thus we are left with two, generalised coordinates, $x_1$ and $x_2$ which are the positions of the two masses in the diagram above. For the equations of motion, we will also require the masses' velocities, $\dot{x_1}$ and $\dot{x_2}$. Let's define these using sympy

In [2]:
x1 = sy.Function('x1')(t)
x2 = sy.Function('x2')(t)
dx1dt = x1.diff(t)
dx2dt = x2.diff(t)

---

## Potential Energy

Let's introduce the potential energy, $V$. The potential energy is the sum of the energy stored in each spring. For springs 1 and 3, the potential energies are $V_1 = \frac{1}{2} k_1 x_1 ^2 $ and $V_3 = \frac{1}{2} k_3 x_2 ^2 $. For spring 2 (the middle spring), the displacement is given by the difference between $x_1$ and $x_3$, and thus the potential energy is $V_2 = \frac{1}{2} k_2 \left( x_1 - x_2\right) ^2 $. In sympy then:

In [3]:
V = sy.Rational(1, 2) * (k1*x1**2 + k3*x2**2 + k2*(x2-x1)**2)
sy.pprint(V)

     2                         2        2   
k₁⋅x₁ (t)   k₂⋅(-x₁(t) + x₂(t))    k₃⋅x₂ (t)
───────── + ──────────────────── + ─────────
    2                2                 2    


---

## General equation of motion for (small) oscillations (around equilibrium)

The general equation of motion for (small) oscillations (around equilibrium) can be formulated in a compact form as a single matrix equation
$${\bf M  q''} = -{\bf K q },$$
where $\bf{q}$ is the vector of generalised coordinates and $\bf{q''}$ denotes the second derivative with respect to time.

The element $K_{jk}$ of the matrix $\bf{K}$ is defined as
$$K_{jk} = K_{kj} = \frac{\partial^2 V}{\partial q_j \partial q_k},$$
where $V$ is the potential energy. 

The element $M_{jk}$ of the matrix $\bf{M}$ is defined as
$$M_{jk} = M_{kj} = \frac{\partial^2 T}{\partial \dot q_j \partial \dot q_k}.$$

The element $q_j(t)$ of the solution of the equation of motion $\bf{q}(t)$  has the general form $A_j e^{i\omega t}$ where $A_{j}$ is the oscillation amplitude and $\omega$ the frequency.

Inserting this in the equation of motion we obtain
$$\omega^2 {\bf M  A} = {\bf K A }.$$
where the column matrix $\bf{A}$ has elements $A_j$.  

For this case, each element of the $\bf{K}$ matrix is given by the second derivative with respect to the relevant coordinate ($x_1$ and/or $x_2$).

In [6]:
K11 = V.diff(x1, x1)
K22 = V.diff(x2, x2)
K12 = V.diff(x1, x2)
K = sy.Matrix([[K11, K12], [K12, K22]])
sy.pprint(K)

⎡k₁ + k₂    -k₂  ⎤
⎢                ⎥
⎣  -k₂    k₂ + k₃⎦


For this case the $\bf{M}$ matrix is trivial. This is not always the case, and in general one must calculate the element $M_{jk}$ of the matrix $\bf{M}$ as indicated above.

In [7]:
M = sy.Matrix([[m1, 0], [0, m2]])
sy.pprint(M)

⎡m₁  0 ⎤
⎢      ⎥
⎣0   m₂⎦


---

## Normal modes
In order to find the normal modes we first define numerical functions for the K and M matrices using `lambdify`:

In [9]:
K_func = sy.lambdify((k1, k2, k3), K)
M_func = sy.lambdify((m1, m2), M)

`K_func` takes as input arguments the constants of the three springs and returns the matrix `K`, as defined above. Here below, we input "1" (N/m)  for all three spring constants and return the result into the variable `Springs`. Similarly, the function `M_func`, takes as input arguments the masses and returns the matrix `M`.  Here below, we input "2" (Kg) for both masses and return the result into the variable `Masses`.

In [11]:
Springs = K_func(1, 1, 1)
Masses = M_func(2, 2)
print(Springs)

[[ 2 -1]
 [-1  2]]


In [12]:
print(Masses)

[[2 0]
 [0 2]]


Finally, we use the `eig` function to determine the normal modes

In [13]:
n_modes = eig(Springs, Masses)

The latter solves the generalised eigenvalue problem 
$${\bf K}{\bf A}_j = \omega_j^2 {\bf M}{\bf A}_j,$$

where $\{\omega_j, {\bf A_j}\}$  is the pair of the generalised eigensolutions. It looks like a standard eigenvalue problem except that instead the identity we have the "Masses" matrix. 

`n_modes` is a list of two arrays. The first array in the list contains the eigenvalues

In [14]:
print(n_modes[0])

[0.5+0.j 1.5+0.j]


The second array in the list contains the corresponding eigenvectors

In [15]:
print(n_modes[1])

[[ 0.70710678 -0.70710678]
 [ 0.70710678  0.70710678]]


Compare these results with what we obtained in class/example in the book.